<a href="https://colab.research.google.com/github/matheuslemesam/Bird_Detection-DL/blob/Fine-Tuning/Bird_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bird Detection, projeto de detecção de espécies de pássaros utilizando Redes Convolucionais. UnB/FCTE - 2025.2 - Professor Vinicius Rispoli**

**Para começar o projeto, definimos possíveis arquiteturas mais promissoras: entre elas YOLO, U-Net, EfficientNetV2-L, EfficientNet-B4, ConvNeXt-Tiny. A EfficientNetV2-L foi a que mais se destacou pelo fato de ter uma precisão melhor, treinamento mais robusto e uma melhor tecnologia de detecção.**

### **Importação do Dataset do Google Drive: montar o Google Drive para acessar o dataset de pássaros.**

Começamos iniciando a GPU e vendo se foi iniciada.

In [8]:
!nvidia-smi

Thu Oct  9 23:56:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
from google.colab import drive
import os
from pathlib import Path

# Montar o Google Drive
drive.mount('/content/drive')

# Definir caminhos para Google Colab
base_path = '/content/drive/MyDrive/Dataset_Aves'
dataset_path = os.path.join(base_path, 'original')
output_path = os.path.join(base_path, 'augmentation')

# Verificar se o dataset existe
if os.path.exists(dataset_path):
    print(f"Dataset encontrado em: {dataset_path}")

    # Listar as espécies disponíveis
    species = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    print(f"Espécies encontradas ({len(species)}): {species}")

    # Contar imagens por espécie
    total_images = 0
    for specie in species:
        specie_path = os.path.join(dataset_path, specie)
        img_count = len([f for f in os.listdir(specie_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"   {specie}: {img_count} imagens")
        total_images += img_count

    print(f"\nTotal de imagens no dataset: {total_images}")
    print(f"Pasta de entrada: {dataset_path}")
    print(f"Pasta de saída: {output_path}")

else:
    print(f"Dataset não encontrado em: {dataset_path}")
    print("Verifique o caminho do dataset")

MessageError: Error: credential propagation was unsuccessful

## **Data Augmentation: aumentar os dados de forma artificial, neste caso com rotações, translações e espelhamento.**

Para o data augmentation, primeiramente importamos as bibliotecas necessárias, Pytorch e Keras.

In [ ]:
!pip install -U torch torchvision torchaudio
!pip install -U keras scikit-learn

In [ ]:
import os
import random
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm # Para uma bela barra de progresso!


source_dir = dataset_path # Onde estão as imagens originais
target_dir = output_path # Onde as novas imagens serão salvas
target_images_per_class = 1000

# Nota: NÃO usamos ToTensor() ou Normalize() aqui, pois queremos salvar
# as imagens como arquivos .jpg/.png, não como tensores.
augmentation_transform = transforms.Compose([
    transforms.RandomResizedCrop(299, scale=(0.7, 1.0)), # Corta e redimensiona de forma mais agressiva
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30), # Aumenta a rotação
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), shear=10), # Pequenas distorções
])

print(f"Iniciando o processo de aumento de dados.")
print(f"Diretório de origem: {source_dir}")
print(f"Diretório de destino: {target_dir}\n")

# Garante que o diretório de destino principal exista
os.makedirs(target_dir, exist_ok=True)

# Lista todas as classes (que são os subdiretórios)
class_names = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

for class_name in class_names:
    source_class_path = os.path.join(source_dir, class_name)
    target_class_path = os.path.join(target_dir, class_name)

    # Cria o subdiretório de destino para a classe
    os.makedirs(target_class_path, exist_ok=True)

    # Lista todas as imagens originais
    original_images = [f for f in os.listdir(source_class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    num_original = len(original_images)

    print(f"Processando classe: '{class_name}'")
    print(f"  Encontradas {num_original} imagens originais.")

    # --- Passo A: Copiar as imagens originais ---
    print(f"  Copiando originais para o destino...")
    for img_name in tqdm(original_images, desc=f"Copiando {class_name}"):
        source_img_path = os.path.join(source_class_path, img_name)
        target_img_path = os.path.join(target_class_path, img_name)
        img = Image.open(source_img_path).convert("RGB")
        img.save(target_img_path)

    # --- Passo B: Gerar novas imagens ---
    num_to_generate = target_images_per_class - num_original
    if num_to_generate <= 0:
        print(f"  A classe '{class_name}' já possui {num_original} imagens. Nenhuma imagem nova será gerada.\n")
        continue

    print(f"  Gerando {num_to_generate} novas imagens via augmentation...")
    for i in tqdm(range(num_to_generate), desc=f"Gerando {class_name}"):
        # Escolhe uma imagem original aleatória como base
        random_image_name = random.choice(original_images)
        base_image_path = os.path.join(source_class_path, random_image_name)

        # Abre a imagem com a biblioteca PIL
        with Image.open(base_image_path).convert("RGB") as img:
            # Aplica a transformação de augmentation
            augmented_img = augmentation_transform(img)

            # Salva a nova imagem com um nome único
            new_image_name = f"aug_{i+1}_{random_image_name}"
            save_path = os.path.join(target_class_path, new_image_name)
            augmented_img.save(save_path)
    print(f"  Classe '{class_name}' finalizada.\n")

print("="*50)
print("PROCESSO DE DATA AUGMENTATION CONCLUÍDO!")
print(f"O novo dataset está pronto em: {target_dir}")
print("="*50)

## **Treinamento do modelo**

Começamos especificando qual framework de Deep Learning o Keras usará, neste caso o PyTorch.

In [3]:
import os
os.environ["KERAS_BACKEND"] = "torch"

Após isto importamos o keras e pytorch para a parte das Redes Neurais e numpy, matplotlib, panda e scikit-learn para visualizações de dados.

In [4]:
import keras
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Agora, fazemos um debug das importações e tomada de decisão: utilizar GPU caso tenha, caso contrário, utilizar a CPU.

In [5]:
print(f"Versão do Keras: {keras.__version__}") # Printa a versão do Keras

print(f"Keras está usando o backend: {keras.backend.backend()}") # Printa o framework utilizado pelo Keras
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Verifica se tem GPU para utilizar

print(f"Usando o dispositivo: {device}") # Mostra a GPU/CPU que será utilizada

Versão do Keras: 3.10.0
Keras está usando o backend: torch
Usando o dispositivo: cuda


Agora, importamos a arquitetura do modelo EfficientNetV2 com a rede neural construida camada por camada e o modificamos para utilizarmos apenas a effnetv2_l

In [6]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021).
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""

import torch
import torch.nn as nn
import math

__all__ = ['effnetv2_l']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)


class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=1000, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(3, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

Fine Tuning


In [1]:
import torch
import torch.nn as nn
from torchvision import models

Aqui estamos carregando a estrutura EfficientNetV2-L e adaptando o dataset

In [7]:
model = effnetv2_l(num_classes=1000)

num_classes = len(species)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features, num_classes)

NameError: name 'species' is not defined

Aqui estamos congelando as camadas convulacionais e treinando apenas a última camada que é a de classificação

In [ ]:
for name, param in model.features.named_parameters():
    param.requires_grad = False

Verifica se existe GPU disponível

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Aqui estamos defininindo as funções de perda e otimização


In [ ]:
perda = nn.CrossEntropyLoss()
otimizacao = torch.optim.Adam(model.classifier.parameters(), lr=1e-4)